# Demo: Extracción de Beneficiarios Finales

Este cuaderno muestra cómo descargar 50 PDFs públicos de beneficiarios finales, ejecutar OCR para extraer RUT y porcentajes, y consolidar los resultados.

## 1. Dependencias
Instala las librerías necesarias si aún no están presentes en tu entorno (ejecuta solo una vez).

In [ ]:
# !pip install --quiet pymupdf pillow pytesseract requests pandas

## 2. Configuración de carpetas
Actualiza las rutas según tu estructura en Databricks o local.

In [ ]:
from pathlib import Path

from beneficiarios_finales_pipeline import (
    download_public_beneficiary_pdfs,
    extract_beneficiarios_from_pdf,
    consolidar_resultados,
)

PROJECT_ROOT = Path.cwd()
PDF_FOLDER = PROJECT_ROOT / "data" / "pdfs_publicos"
IMAGES_FOLDER = PROJECT_ROOT / "data" / "imagenes"
PDF_FOLDER.mkdir(parents=True, exist_ok=True)
IMAGES_FOLDER.mkdir(parents=True, exist_ok=True)

## 3. Descarga de PDFs públicos
El siguiente bloque intenta recolectar hasta 50 PDFs desde la Comisión para el Mercado Financiero (CMF) de Chile.
Puedes ajustar `catalog_urls` si deseas otras fuentes públicas.

In [ ]:
sample_pdfs = download_public_beneficiary_pdfs(
    destination=PDF_FOLDER,
    limit=50,
    catalog_urls=None,  # Reemplaza por una lista de URLs si tienes un catálogo específico.
)
len(sample_pdfs), sample_pdfs[:3]

## 4. Extracción OCR
Convertimos cada PDF en imágenes, aplicamos OCR y consolidamos los resultados en un `DataFrame`.

In [ ]:
resultados = []
for pdf_path in sample_pdfs:
    resultados.extend(
        extract_beneficiarios_from_pdf(
            pdf_path=pdf_path,
            output_root=IMAGES_FOLDER,
            dpi=200,
            lang="spa",
        )
    )

df_resultados = consolidar_resultados(resultados)
df_resultados.head()

## 5. Exportar y validar
En esta sección puedes guardar los resultados en Databricks o validarlos manualmente.

In [ ]:
# Ejemplo de exportación local.
# df_resultados.to_csv(PROJECT_ROOT / "data" / "resultado_beneficiarios.csv", index=False)
df_resultados.sample(min(len(df_resultados), 10))